In [1]:
import pandas as pd
import numpy as np
import os
import pathlib
from tqdm import tqdm
import warnings
import itertools
warnings.filterwarnings("ignore")

# machine learning models
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
from sklearn.metrics import roc_curve

maindir = "/Users/hieunguyen/data"
outdir = "/Users/hieunguyen/data/outdir"

PROJECT = "WGS_features"
data_version = "20240725"
output_version = "20240730"
path_to_main_input = os.path.join(maindir, PROJECT, data_version)
path_to_main_output = os.path.join(outdir, PROJECT, data_version, output_version)

path_to_01_output = os.path.join(path_to_main_output, "01_output")
os.makedirs(path_to_01_output, exist_ok=True)

metadata = pd.read_excel(os.path.join(path_to_main_input, "metadata", "metadata_WGS_20240725.xlsx"))
sample_list = {}
for input_label in metadata["Label"].unique():
    sample_list[input_label] = metadata#.loc[metadata["Label"] == input_label, "SampleID"].tolist()

all_features = ["GWfeature_EM",
                "GWfeature_flen", 
                "GWfeature_Nucleosome",
                "GWfeature_CNA",
                "GWfeature_flen_ratio"]

unseen_runs = ["R5434", "R5451", "R5470", "R5484"]
discovery_runs = list(set(metadata["Run"].unique()) - set(unseen_runs))

feature_files = [item for item in pathlib.Path(path_to_01_output).glob("csv/*.csv")]
feature_file_names = [item.name.replace(".csv", "") for item in feature_files]
feature_files = dict(zip(feature_file_names, feature_files))

#####-----------------------------------------------------------------#####
##### DEFINE MODELS
#####-----------------------------------------------------------------#####
models = {
    'SVC': {
        'model': SVC(),
        'params': {
            'C': [0.1, 1],
            'kernel': ['rbf'],
            'probability': [True]
        }
    },
    'RandomForest': {
        'model': RandomForestClassifier(),
        'params': {
            'n_estimators': [10, 50, 100],
            'max_depth': [30,40, 50]
        }
    },
    'LogisticRegression': {
        'model': LogisticRegression(max_iter=1000),
        'params': {
            'C': [0.1, 1, 5, 10],
            'solver': ['liblinear', 'lbfgs']
        }
    },
    'KNeighbors': {
        'model': KNeighborsClassifier(),
        'params': {
            'n_neighbors': [3, 5, 7, 9],
            'weights': ['uniform', 'distance']
        }
    },
    'DecisionTree': {
        'model': DecisionTreeClassifier(),
        'params': {
            'max_depth': [10, 20, 30, 40],
            'criterion': ['gini', 'entropy']
        }
    },
    'XGBoost': {
        'model': XGBClassifier(use_label_encoder=False, eval_metric='mlogloss'),
        'params': {
            'n_estimators': [50, 100, 200],
            'max_depth': [3, 5, 7],
            'learning_rate': [0.01, 0.1, 0.2],
            'probability': [True]
        }
    }
}

In [2]:
countdf = metadata.groupby("Label")["SampleID"].count().reset_index()

In [3]:
metadata = metadata[metadata["Label"].isin(countdf[countdf["SampleID"] >= 20].Label.unique())]

In [4]:
unseen_samples = dict()
training_samples = dict()
for input_class in metadata.Label.unique():
    tmp_metadata = metadata[metadata["Label"] == input_class].sort_values(by = "Run")
    sampling_samples = tmp_metadata.tail(np.floor(0.20*tmp_metadata.shape[0]).astype(int)).SampleID.to_list()
    if len(sampling_samples) != 0:
        unseen_samples[input_class] = sampling_samples
        training_samples[input_class] = [item for item in metadata[metadata["Label"] == input_class].SampleID.unique() if item not in sampling_samples]

In [5]:
#####-----------------------------------------------------------------#####
##### Prepare data for machine learning models
#####-----------------------------------------------------------------######

def train(feature_name, 
          training_samples,
          unseen_samples,
          cancer_class, 
          models,
          model_name = "XGBoost", 
          control_class = "Control", 
          num_cv = 5):
    # feature_name = "GWfeature_flen_ratio"
    # cancer_class = "CRC"
    # control_class = "Control"
      
    maindf = pd.read_csv(feature_files[feature_name], index_col = [0])
    maindf = maindf.loc[[item for item in list(maindf.index) if "X" not in str(item) and "Y" not in str(item)], :]
    if cancer_class == "all":
      selected_training_samples = list(itertools.chain(*training_samples.values()))
      selected_unseen_samples = list(itertools.chain(*unseen_samples.values()))
    else:
      selected_training_samples = training_samples[cancer_class] + training_samples[control_class]
      selected_unseen_samples = unseen_samples[cancer_class] + unseen_samples[control_class]
      
    Xtrain = maindf[selected_training_samples].to_numpy().T
    sample_labels = [metadata[metadata["SampleID"] == item].Label.values[0] for item in selected_training_samples]
    ytrain = [1 if item != "Control" else 0 for item in sample_labels]     
    
    Xtest = maindf[selected_unseen_samples].to_numpy().T
    sample_labels = [metadata[metadata["SampleID"] == item].Label.values[0] for item in selected_unseen_samples]
    ytest = [1 if item != "Control" else 0 for item in sample_labels]     
    
    
    model_info = models[model_name]
    clf = GridSearchCV(model_info['model'], model_info['params'], cv=5, scoring='accuracy')
    clf.fit(Xtrain, ytrain)
    best_models = clf.best_estimator_
    cv_scores = cross_val_score(best_models, Xtrain, ytrain, cv=num_cv, scoring='accuracy')
    best_models.fit(Xtrain, ytrain)
    
    ytrain_proba = best_models.predict_proba(Xtrain)[:, 1]
    fpr, tpr, thresholds = roc_curve(ytrain, ytrain_proba, pos_label = 1)
    roc_df = pd.DataFrame({
        'FPR': fpr,
        'TPR': tpr,
        'Threshold': thresholds
    })
    # print(roc_df)
    roc_df = roc_df[roc_df["FPR"] <= 0.05]
    thres = roc_df[roc_df["TPR"] == roc_df["TPR"].max()]["Threshold"].to_list()[0]
    ytest_proba = best_models.predict_proba(Xtest)[:, 1]

    unseendf = pd.DataFrame(data = selected_unseen_samples, columns = ["SampleID"])
    unseendf["prediction"] = [1 if item >= thres else 0 for item in ytest_proba]
    unseendf["true_label"] = ytest
    spec = unseendf[(unseendf["true_label"] == 0) & (unseendf["prediction"] == 0)].shape[0]/unseendf[unseendf["true_label"] == 0].shape[0]
    sen = unseendf[(unseendf["true_label"] == 1) & (unseendf["prediction"] == 1)].shape[0]/unseendf[unseendf["true_label"] == 1].shape[0]
    unseen_acc = accuracy_score(ytest, unseendf["prediction"].to_list())
    return best_models, cv_scores, unseen_acc, unseendf, sen, spec

In [6]:
training_resdf = pd.DataFrame()
feature_name = "GWfeature_flen_ratio"
model_name = "XGBoost"
for model_name in models.keys(): 
    for feature_name in all_features:   
        # for cancer_class in tqdm([item for item in training_samples.keys() if item != "Control"]):
        for cancer_class in ["all"]:
            num_cv = 5

            best_models, cv_scores, unseen_acc, unseendf, sen, spec = train(
                feature_name = feature_name, 
                cancer_class = cancer_class, 
                training_samples = training_samples,
                unseen_samples = unseen_samples,
                models = models,
                model_name = model_name,
                control_class = "Control", 
                num_cv = num_cv)


            tmpdf = pd.DataFrame(data = [feature_name], columns = ["Feature"])
            tmpdf["cancer_class"] = cancer_class
            tmpdf["model"] = model_name
            tmpdf["num_cv"] = num_cv
            tmpdf["best_model"] = best_models.get_params()
            tmpdf["mean_cv_scores"] = np.mean(cv_scores)
            tmpdf["median_cv_scores"] = np.median(cv_scores)
            tmpdf["range_cv_score"] = "{:.2f}:{:.2f}".format(np.min(cv_scores), np.max(cv_scores))
            tmpdf["unseen_acc"] = unseen_acc
            tmpdf["sensitivity"] = sen
            tmpdf["specificity"] = spec
            training_resdf = pd.concat([training_resdf, tmpdf], axis = 0)

In [7]:
training_resdf

,Feature,cancer_class,model,num_cv,best_model,mean_cv_scores,median_cv_scores,range_cv_score,unseen_acc,sensitivity,specificity
0,GWfeature_EM,all,SVC,5,NaN,0.641198,0.632353,0.59:0.72,0.625000,0.259259,0.965517
0,GWfeature_flen,all,SVC,5,NaN,0.664631,0.654412,0.58:0.77,0.607143,0.246914,0.942529
0,GWfeature_Nucleosome,all,SVC,5,NaN,0.510981,0.510949,0.51:0.51,0.511905,0.000000,0.988506
0,GWfeature_CNA,all,SVC,5,NaN,0.603113,0.580882,0.55:0.72,0.636905,0.629630,0.643678
0,GWfeature_flen_ratio,all,SVC,5,NaN,0.594365,0.562044,0.52:0.71,0.601190,0.234568,0.942529
0,GWfeature_EM,all,RandomForest,5,NaN,0.701106,0.737226,0.58:0.77,0.696429,0.493827,0.885057
0,GWfeature_flen,all,RandomForest,5,NaN,0.580904,0.642336,0.26:0.72,0.696429,0.469136,0.908046
0,GWfeature_Nucleosome,all,RandomForest,5,NaN,0.619075,0.620438,0.45:0.73,0.636905,0.407407,0.850575
0,GWfeature_CNA,all,RandomForest,5,NaN,0.601760,0.610294,0.55:0.64,0.565476,0.123457,0.977011
0,GWfeature_flen_ratio,all,RandomForest,5,NaN,0.560595,0.532847,0.49:0.69,0.571429,0.370370,0.758621
